In [59]:
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split


In [60]:
data= pd.read_csv("C:/Users/Kareem/Desktop/annclassification/annclassification/Churn_Modelling.csv")

In [61]:
data.head(5)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [62]:
print(type(data["Geography"]))

print(type(data[["Geography"]]))
encoder=OneHotEncoder()
geography_encoded=encoder.fit_transform(data[["Geography"]]).toarray()

<class 'pandas.core.series.Series'>
<class 'pandas.core.frame.DataFrame'>


In [63]:
geography_encoded_pd=pd.DataFrame(geography_encoded,columns=encoder.get_feature_names_out(["Geography"]))

In [64]:
X=pd.concat([data.drop(["RowNumber","Geography","CustomerId","Surname"],axis=1),geography_encoded_pd],axis=1)

In [65]:
y=X["Exited"]
X=X.drop(["Exited"],axis=1)
encoder=LabelEncoder()
X["Gender"]=encoder.fit_transform(X['Gender'])

X.head(5)


,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0.0,0.0,1.0


In [66]:
X_train, X_test, y_train, y_test=train_test_split(X,y,test_size=0.2,shuffle=True,random_state=42)

In [67]:
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard

In [ ]:
log_dir="D:Nlp_course/log2/fit"
tensorBoard_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)
EarlyStopping_callback=EarlyStopping(monitor='accuracy'
,patience=10,restore_best_weights=True)

In [ ]:

def model(neurons=32,layers=2):
    model= Sequential()
    model.add(Dense(neurons,activation="relu",input_shape=(X.shape[1],)))
    for _ in range(layers-1):
      model.add(Dense(neurons,activation='relu'))
    model.add(Dense(1,activation='sigmoid'))
    model.compile(loss="binary_crossentropy",metrics=['accuracy'],optimizer='Adam')
    return model

from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV
kera=KerasClassifier(build_fn=model,epochs=100,batch_size=10,callbacks=[EarlyStopping_callback, tensorBoard_callback])

param_grid={
  'model__neurons': [16, 32, 64, 128],
    'model__layers': [1, 2],
   'epochs':[30,50]
}    

      
grid=GridSearchCV(estimator=kera,param_grid=param_grid,cv=3)
grid.fit(X_train,y_train)

print(grid.best_score_,grid.best_params_)

d:\NLP_COURSE\nlp\lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
d:\NLP_COURSE\nlp\lib\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/30
534/534 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.6921 - loss: 113.5219
Epoch 2/30
534/534 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.6854 - loss: 42.7915
Epoch 3/30
534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6855 - loss: 41.9615
Epoch 4/30
534/534 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.6816 - loss: 43.4197
Epoch 5/30
534/534 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.6779 - loss: 50.5831
Epoch 6/30
534/534 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.6717 - loss: 36.3805
Epoch 7/30
534/534 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.6878 - loss: 35.7212
Epoch 8/30
534/534 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.6752 - loss: 38.4347
Epoch 9/30
534/534 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.6779 - loss: 32.4444
Epoch 10/30
534/534 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.6895 - loss: 39.6805
Epoch 11/30
534/534 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.6839 - loss: 35.2506
Epoch 12/30
534/534 ━━━━━━━━━━━━━━━━━━━━

d:\NLP_COURSE\nlp\lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
d:\NLP_COURSE\nlp\lib\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/30
534/534 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.6623 - loss: 481.7709 
Epoch 2/30
534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6764 - loss: 63.5576
Epoch 3/30
534/534 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.6698 - loss: 51.0494
Epoch 4/30
534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6831 - loss: 33.6188
Epoch 5/30
534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6760 - loss: 37.7033
Epoch 6/30
534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6758 - loss: 32.6138
Epoch 7/30
534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6762 - loss: 30.9612
Epoch 8/30
534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6852 - loss: 30.8451
Epoch 9/30
534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6788 - loss: 30.7697
Epoch 10/30
534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6934 - loss: 34.6244
Epoch 11/30
534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6810 - loss: 28.7199
Epoch 12/30
534/534 ━━━━━━━━━━━━━━━━━━━

d:\NLP_COURSE\nlp\lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
d:\NLP_COURSE\nlp\lib\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


...

Epoch 1/30
534/534 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.6610 - loss: 443.8383 
Epoch 2/30
534/534 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.6720 - loss: 120.2279
Epoch 3/30
534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6735 - loss: 146.8609
Epoch 4/30
534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6756 - loss: 107.9957
Epoch 5/30
534/534 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.6707 - loss: 122.9180
Epoch 6/30
534/534 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.6769 - loss: 88.2250 
Epoch 7/30
534/534 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.6737 - loss: 113.2200
Epoch 8/30
534/534 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.6713 - loss: 96.7917
Epoch 9/30
534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6769 - loss: 87.2317
Epoch 10/30
534/534 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.6749 - loss: 99.5750
Epoch 11/30
534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6837 - loss: 90.9731 
Epoch 12/30
534/534 ━━━━━━━━━━━━

d:\NLP_COURSE\nlp\lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
d:\NLP_COURSE\nlp\lib\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/30
534/534 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.6297 - loss: 2246.8313
Epoch 2/30
534/534 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.6944 - loss: 86.9954
Epoch 3/30
534/534 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.6826 - loss: 77.0787
Epoch 4/30
534/534 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.6894 - loss: 59.8192
Epoch 5/30
534/534 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.6826 - loss: 71.6516
Epoch 6/30
534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6826 - loss: 58.2033
Epoch 7/30
534/534 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.6740 - loss: 73.9370
Epoch 8/30
534/534 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.6736 - loss: 70.6768
Epoch 9/30
534/534 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.6845 - loss: 61.6567
Epoch 10/30
534/534 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.6820 - loss: 70.5816
Epoch 11/30
534/534 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.6807 - loss: 64.0454
Epoch 12/30
534/534 ━━━━━━━━━━━━━━━━━━━

d:\NLP_COURSE\nlp\lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
d:\NLP_COURSE\nlp\lib\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/30
534/534 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.6593 - loss: 293.7637
Epoch 2/30
534/534 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.6784 - loss: 104.2998
Epoch 3/30
534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6820 - loss: 137.0059
Epoch 4/30
534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6809 - loss: 146.7779
Epoch 5/30
534/534 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.6899 - loss: 113.6971
Epoch 6/30
534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6861 - loss: 144.0817
Epoch 7/30
534/534 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.6891 - loss: 111.4645
Epoch 8/30
534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6842 - loss: 135.9102
Epoch 9/30
534/534 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.6844 - loss: 111.9957
Epoch 10/30
534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6889 - loss: 90.1512
Epoch 11/30
534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6942 - loss: 88.8193
Epoch 12/30
534/534 ━━━━━━━━━━━━

In [ ]:
best_keras_model = grid.best_estimator_.model_
best_keras_model.save('best_model.keras')

In [ ]:
%load_ext tensorboard 

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [ ]:
%tensorboard --logdir D:Nlp_course/log2/fit



Reusing TensorBoard on port 6006 (pid 3856), started 1:52:25 ago. (Use '!kill 3856' to kill it.)